# Load Necessary Stuff

In [ ]:
!pip install Pillow==9.4.0

In [ ]:
!unzip /content/test.zip -d /content/

In [ ]:
from PIL import Image
Image.open('/content/test/220812_GFP-AHPC_C_GFAP_F10_DAPI_ND1_20x.tiff')

In [ ]:
%matplotlib inline
import torch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from matplotlib.collections import PatchCollection
import matplotlib.patches as patches
import os

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path="/content/best.pt")
model.conf = 0.20  # NMS confidence threshold
model.iou = 0.20  # NMS IoU threshold

In [ ]:
img = []
for i in os.listdir('/content/test'):
  if '.tiff' in i:
    img.append(i)

In [ ]:
# calculate MSE (Incorrect)
from sklearn.metrics import mean_absolute_error

# Quick Overview Result (Raw MAE, Raw ACP, Inside box MAE, Inside box ACP)

In [ ]:
#raw mae
y_pred = []
y_true = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  y_pred.append(len(pred_result.pandas().xyxy[0]))
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  y_true.append(len(true_result))

mean_absolute_error(y_true, y_pred)

In [ ]:
#raw acp
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]
  y_pred = len(pred_result)

  cnt_num += 1


  if abs(y_pred-y_true) <= 0.05 * y_true:
    satisfy_acp += 1
print(satisfy_acp/cnt_num)

In [ ]:
def in_box(pred_result,x,y):
  return pred_result[(pred_result['xmin'] <= x) & (pred_result['xmax'] >= x) & (pred_result['ymin'] <= y) & (pred_result['ymax'] >= y)]

In [ ]:
# inside_box MAE
inside_box = []

for i in range(len(img)):

  # Load Pred & True Result
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))


  pred_result = pred_result.pandas().xyxy[0]
  true_result = true_result[['X','Y']].values.tolist()
  cnt = 0

  # copy pred_result
  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  inside_box.append(abs(cnt-len(true_result)))
sum(inside_box)/len(inside_box)

In [ ]:
#inside box acp
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]


  # true cell is the # of cell in bounding box
  true_cell = 0

  # copy pred_result
  pred_temp = pred_result.copy()
  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      true_cell += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])


  y_pred = true_cell
  cnt_num += 1


  if abs(y_pred-y_true) <= 0.05 * y_true:
    satisfy_acp += 1
print(satisfy_acp/cnt_num)

# Detailed Low Mid High Raw MAE

In [ ]:
# Raw Low
y_pred = []
y_true = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  if len(true_result) <= 45:
    y_true.append(len(true_result))
    y_pred.append(len(pred_result.pandas().xyxy[0]))

mean_absolute_error(y_true, y_pred)

In [ ]:
# Raw Mid
y_pred = []
y_true = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  if len(true_result) <= 310 and len(true_result) > 45:
    y_true.append(len(true_result))
    y_pred.append(len(pred_result.pandas().xyxy[0]))

mean_absolute_error(y_true, y_pred)

In [ ]:
# Raw High
y_pred = []
y_true = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  if len(true_result) > 310:
    y_true.append(len(true_result))
    y_pred.append(len(pred_result.pandas().xyxy[0]))

mean_absolute_error(y_true, y_pred)

# Detailed Low Mid High Raw ACP

In [ ]:
#raw low acp
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]
  y_pred = len(pred_result)


  if len(true_result) <= 45:
    cnt_num += 1
    if abs(y_pred-y_true) <= 0.05 * y_true:
      satisfy_acp += 1
print(satisfy_acp/cnt_num)
print(satisfy_acp, cnt_num)

In [ ]:
#raw mid acp
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]
  y_pred = len(pred_result)


  if len(true_result) <= 310 and len(true_result) > 45:
    cnt_num += 1

    if abs(y_pred-y_true) <= 0.05 * y_true:
      satisfy_acp += 1
print(satisfy_acp/cnt_num)
print(satisfy_acp, cnt_num)

In [ ]:
#raw high acp
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]
  y_pred = len(pred_result)


  if len(true_result) > 310:
    cnt_num += 1

    if abs(y_pred-y_true) <= 0.05 * y_true:
      satisfy_acp += 1
print(satisfy_acp/cnt_num)
print(satisfy_acp, cnt_num)

In [ ]:
# def box_transformation(location, height, width):
#   x, y = location
#   h = height / 2
#   w = width / 2

#   pt1 = (max(0,x-w),max(0,y-h))
#   pt2 = (min(x+h,600),min(800,y+h))
#   pt3 = (min(x+h,600),max(0,y-h))
#   pt4 = (max(0,x-w,min(800,y+h)))

#   return [x-w,y-h,x+w,y+h]

In [ ]:
# from torchvision.ops import box_iou

In [ ]:
# accuracy = []

# for i in range(len(img)):
#   pred_result = model('/content/test/' + img[i])
#   true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))

#   true_result = true_result.sort_values(by=['X', 'Y'])[['X','Y']].values.tolist()
#   pred_xyxy = pred_result.pandas().xyxy[0].sort_values(by=['xmin', 'ymin'])[['xmin','ymin','xmax','ymax']].values.tolist()


#   true_xyxy = []
#   for i in true_result:
#     true_xyxy.append(box_transformation(i, 10,10))

#   cnt = 0

#   size = len(true_xyxy) if len(true_xyxy) < len(pred_xyxy) else len(pred_xyxy)

#   for i in range(size):
#     if sum(box_iou(torch.tensor(pred_xyxy),torch.tensor(true_xyxy))[i] > 0.1) > 0:
#       cnt += 1
#   accuracy.append(cnt/len(true_xyxy))
#   print(i)

In [ ]:
# # IOU Accuracy
# sum(accuracy)/len(accuracy)

In [ ]:
# Annotation inside the box approach

In [ ]:
pred_result = model('/content/test/' + img[10])
true_result = pd.read_csv('/content/test/' + img[10].replace('.tiff','.csv'))

In [ ]:
pred_result = pred_result.pandas().xyxy[0].sort_values(by=['xmin', 'ymin'])[['xmin','ymin','xmax','ymax']]
true_result = true_result.sort_values(by=['X', 'Y'])[['X','Y']].values.tolist()

In [ ]:
true_result

In [ ]:
def in_box(pred_result,x,y):
  return pred_result[(pred_result['xmin'] <= x) & (pred_result['xmax'] >= x) & (pred_result['ymin'] <= y) & (pred_result['ymax'] >= y)]

In [ ]:
cnt = 0
for i in true_result:
  bounding_box = in_box(pred_result,i[0],i[1])
  if len(bounding_box) > 0:
    cnt += 1
    pred_result = pred_result.drop(index=bounding_box.index[0])
print(cnt/len(true_result))

# Average % of inside box annotation per image

In [ ]:
#Overall
inside_box = []

for i in range(len(img)):

  # Load Pred & True Result
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))


  pred_result = pred_result.pandas().xyxy[0]
  true_result = true_result[['X','Y']].values.tolist()
  cnt = 0

  # copy pred_result
  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  inside_box.append(cnt/len(true_result))
sum(inside_box)/len(inside_box)

In [ ]:
# Low %
inside_box = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  pred_result = pred_result.pandas().xyxy[0].sort_values(by=['xmin', 'ymin'])[['xmin','ymin','xmax','ymax']]
  true_result = true_result.sort_values(by=['X', 'Y'])[['X','Y']].values.tolist()
  cnt = 0

  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  if len(true_result) <= 45:
    inside_box.append(cnt/len(true_result))
sum(inside_box)/len(inside_box)

In [ ]:
# Mid %
inside_box = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  pred_result = pred_result.pandas().xyxy[0].sort_values(by=['xmin', 'ymin'])[['xmin','ymin','xmax','ymax']]
  true_result = true_result.sort_values(by=['X', 'Y'])[['X','Y']].values.tolist()
  cnt = 0

  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  if len(true_result) > 45 and len(true_result) <= 310:
    inside_box.append(cnt/len(true_result))
sum(inside_box)/len(inside_box)

In [ ]:
# High %
inside_box = []

for i in range(len(img)):
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  pred_result = pred_result.pandas().xyxy[0].sort_values(by=['xmin', 'ymin'])[['xmin','ymin','xmax','ymax']]
  true_result = true_result.sort_values(by=['X', 'Y'])[['X','Y']].values.tolist()
  cnt = 0

  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  if len(true_result) > 310:
    inside_box.append(cnt/len(true_result))
sum(inside_box)/len(inside_box)

# Detailed Low Mid High MAE Inside Box

In [ ]:
# MAE Inside Box

In [ ]:
# # Overall
# inside_box = []

# for i in range(len(img)):

#   # Load Pred & True Result
#   pred_result = model('/content/test/' + img[i])
#   true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))


#   pred_result = pred_result.pandas().xyxy[0]
#   true_result = true_result[['X','Y']].values.tolist()
#   cnt = 0

#   # copy pred_result
#   pred_temp = pred_result.copy()


#   for i in true_result:
#     bounding_box = in_box(pred_temp,i[0],i[1])
#     if len(bounding_box) > 0:
#       cnt += 1
#       # delete this box so future pts cannot associate with this bounding box
#       pred_temp = pred_temp.drop(index=bounding_box.index[0])
#   inside_box.append(abs(cnt-len(true_result)))
# sum(inside_box)/len(inside_box)

In [ ]:
# Inside Box MAE Low
inside_box = []

for i in range(len(img)):

  # Load Pred & True Result
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))


  pred_result = pred_result.pandas().xyxy[0]
  true_result = true_result[['X','Y']].values.tolist()
  cnt = 0

  # copy pred_result
  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  if len(true_result) <= 45:
    inside_box.append(abs(cnt-len(true_result)))
sum(inside_box)/len(inside_box)

In [ ]:
# Inside Box MAE Mid
inside_box = []

for i in range(len(img)):

  # Load Pred & True Result
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))


  pred_result = pred_result.pandas().xyxy[0]
  true_result = true_result[['X','Y']].values.tolist()
  cnt = 0

  # copy pred_result
  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  if len(true_result) >= 45 and len(true_result) < 310:
    inside_box.append(abs(cnt-len(true_result)))
sum(inside_box)/len(inside_box)

In [ ]:
# Inside Box MAE High
inside_box = []

for i in range(len(img)):

  # Load Pred & True Result
  pred_result = model('/content/test/' + img[i])
  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))


  pred_result = pred_result.pandas().xyxy[0]
  true_result = true_result[['X','Y']].values.tolist()
  cnt = 0

  # copy pred_result
  pred_temp = pred_result.copy()


  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      cnt += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])
  if len(true_result) > 310:
    inside_box.append(abs(cnt-len(true_result)))
sum(inside_box)/len(inside_box)

# Detailed Low Migh High Inside Box ACP

In [ ]:
#ACP OverALL
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]


  # true cell is the # of cell in bounding box
  true_cell = 0

  # copy pred_result
  pred_temp = pred_result.copy()
  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      true_cell += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])


  y_pred = true_cell
  cnt_num += 1


  if abs(y_pred-y_true) <= 0.05 * y_true:
    satisfy_acp += 1
print(satisfy_acp/cnt_num)

In [ ]:
#Inside box ACP Low
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]


  # true cell is the # of cell in bounding box
  true_cell = 0

  # copy pred_result
  pred_temp = pred_result.copy()
  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      true_cell += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])

  if y_true <= 45:
    y_pred = true_cell
    cnt_num += 1

    if abs(y_pred-y_true) <= 0.05 * y_true:
      satisfy_acp += 1
print(satisfy_acp/cnt_num)
print(satisfy_acp, cnt_num)

In [ ]:
#Inside box ACP Mid
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]


  # true cell is the # of cell in bounding box
  true_cell = 0

  # copy pred_result
  pred_temp = pred_result.copy()
  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      true_cell += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])

  if y_true > 45 and y_true <= 310:
    y_pred = true_cell
    cnt_num += 1


    if abs(y_pred-y_true) <= 0.05 * y_true:
      satisfy_acp += 1
print(satisfy_acp/cnt_num)
print(satisfy_acp, cnt_num)

In [ ]:
#Inside boxACP High
satisfy_acp = 0


# count the number of images
cnt_num = 0
for i in range(len(img)):

  true_result = pd.read_csv('/content/test/' + img[i].replace('.tiff','.csv'))
  true_result = true_result[['X','Y']].values.tolist()
  y_true = len(true_result)

  pred_result = model('/content/test/' + img[i])
  pred_result = pred_result.pandas().xyxy[0]


  # true cell is the # of cell in bounding box
  true_cell = 0

  # copy pred_result
  pred_temp = pred_result.copy()
  for i in true_result:
    bounding_box = in_box(pred_temp,i[0],i[1])
    if len(bounding_box) > 0:
      true_cell += 1
      # delete this box so future pts cannot associate with this bounding box
      pred_temp = pred_temp.drop(index=bounding_box.index[0])

  if y_true > 310:
    y_pred = true_cell
    cnt_num += 1


    if abs(y_pred-y_true) <= 0.05 * y_true:
      satisfy_acp += 1
print(satisfy_acp/cnt_num)
print(satisfy_acp, cnt_num)

In [ ]:
# Visualization

In [ ]:
data_path = '/content/test/220909_GFP-AHPC_A_GFAP_F9_DAPI_ND1_20x.csv'
img_path = '/content/test/220909_GFP-AHPC_A_GFAP_F9_DAPI_ND1_20x.tiff'
results = model(img_path)
data = pd.read_csv(data_path)
data = data[['X','Y']]
pred = results.pandas().xyxy[0]
xmin_pred = pred['xmin'].values.tolist()
ymin_pred = pred['ymin'].values.tolist()
height = (pred['ymax'] - pred['ymin']).values.tolist()
width = (pred['xmax'] - pred['xmin']).values.tolist()
img = plt.imread(img_path)


# Counting True prediction Boxes
#######################################################################################
pred_result = model(img_path)
pred_result = pred_result.pandas().xyxy[0][['xmin','ymin','xmax','ymax']]
true_result = pd.read_csv(data_path)
true_result = true_result[['X','Y']].values.tolist()


# true cell is the # of cell in bounding box
true_cell_pred = 0

# copy pred_result
pred_temp = pred_result.copy()
for i in true_result:
  bounding_box = in_box(pred_temp,i[0],i[1])
  if len(bounding_box) > 0:
    true_cell_pred += 1
    # delete this box so future pts cannot associate with this bounding box
    pred_temp = pred_temp.drop(index=bounding_box.index[0])

#######################################################################################




fig, ax = plt.subplots(1,1, figsize=(20,15))
ax.imshow(img)
ax.imshow(img)



ax.scatter(data['X'], data['Y'], color='red', s=70)
for i in range(len(xmin_pred)):
  x = xmin_pred[i]
  y = ymin_pred[i]
  h = height[i]
  w = width[i]
  rect = matplotlib.patches.Rectangle((x, y),w,h, linewidth=2, edgecolor='y', facecolor='none')
  ax.add_patch(rect)



plt.title(f'True:{len(true_result)} Predicted:{true_cell_pred}', fontsize=50)

#plt.show()
plt.savefig('fig.png')

In [ ]:
pred_result = model(img_path)
pred_result = pred_result.pandas().xyxy[0][['xmin','ymin','xmax','ymax']]
true_result = pd.read_csv(data_path)
true_result = true_result[['X','Y']].values.tolist()

In [ ]:
# true cell is the # of cell in bounding box
true_cell_pred = 0

# copy pred_result
pred_temp = pred_result.copy()
for i in true_result:
  bounding_box = in_box(pred_temp,i[0],i[1])
  if len(bounding_box) > 0:
    true_cell_pred += 1
    # delete this box so future pts cannot associate with this bounding box
    pred_temp = pred_temp.drop(index=bounding_box.index[0])

In [ ]:
print(true_cell_pred)

In [ ]:
print(len(true_result))

In [ ]:
pred_result = model(img_path)
pred_result.pandas().xyxy[0]